In [36]:
import tensorflow as tf

In [37]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

tf.config.list_physical_devices('GPU')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10399238918198418418
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1719350068
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8649926068609517911
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [38]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display

In [39]:
# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
  """Returns high confidence keypoints and edges for visualization.

  Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.

  Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:
      * the coordinates of all keypoints of all detected entities;
      * the coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
  """
  keypoints_all = []
  keypoint_edges_all = []
  edge_colors = []
  num_instances, _, _, _ = keypoints_with_scores.shape
  for idx in range(num_instances):
    kpts_x = keypoints_with_scores[0, idx, :, 1]
    kpts_y = keypoints_with_scores[0, idx, :, 0]
    kpts_scores = keypoints_with_scores[0, idx, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > keypoint_threshold, :]
    keypoints_all.append(kpts_above_thresh_absolute)

    for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
      if (kpts_scores[edge_pair[0]] > keypoint_threshold and
          kpts_scores[edge_pair[1]] > keypoint_threshold):
        x_start = kpts_absolute_xy[edge_pair[0], 0]
        y_start = kpts_absolute_xy[edge_pair[0], 1]
        x_end = kpts_absolute_xy[edge_pair[1], 0]
        y_end = kpts_absolute_xy[edge_pair[1], 1]
        line_seg = np.array([[x_start, y_start], [x_end, y_end]])
        keypoint_edges_all.append(line_seg)
        edge_colors.append(color)
  if keypoints_all:
    keypoints_xy = np.concatenate(keypoints_all, axis=0)
  else:
    keypoints_xy = np.zeros((0, 17, 2))

  if keypoint_edges_all:
    edges_xy = np.stack(keypoint_edges_all, axis=0)
  else:
    edges_xy = np.zeros((0, 2, 2))

  # print(keypoints_xy)
  return keypoints_xy, edges_xy, edge_colors


def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return keypoint_locs,image_from_plot

def to_gif(images, fps):
  """Converts image sequence (4D numpy array) to gif."""
  imageio.mimsave('./animation.mp4', images, fps=fps)
  return embed.embed_file('./animation.mp4')

def progress(value, max=100):
  return HTML("""
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  """.format(value=value, max=max))

In [40]:
def to_mp4(images, fps, output_file='output.mp4'):
    """Converts image sequence (4D numpy array) to MP4."""
    height, width, _ = images[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # mp4v 코덱 사용
    video_writer = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    for image in images:
        # 이미지 데이터를 BGR로 변환 (OpenCV는 BGR 포맷을 사용)
        bgr_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        video_writer.write(bgr_image)

    video_writer.release()
    print(f"Video saved as {output_file}")

In [41]:
model_name = "movenet_lightning"

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

In [42]:
# Confidence score to determine whether a keypoint prediction is reliable.
MIN_CROP_KEYPOINT_SCORE = 0.2

def init_crop_region(image_height, image_width):
  """Defines the default crop region.

  The function provides the initial crop region (pads the full image from both
  sides to make it a square image) when the algorithm cannot reliably determine
  the crop region from the previous frame.
  """
  if image_width > image_height:
    box_height = image_width / image_height
    box_width = 1.0
    y_min = (image_height / 2 - image_width / 2) / image_height
    x_min = 0.0
  else:
    box_height = 1.0
    box_width = image_height / image_width
    y_min = 0.0
    x_min = (image_width / 2 - image_height / 2) / image_width

  return {
    'y_min': y_min,
    'x_min': x_min,
    'y_max': y_min + box_height,
    'x_max': x_min + box_width,
    'height': box_height,
    'width': box_width
  }

def torso_visible(keypoints):
  """Checks whether there are enough torso keypoints.

  This function checks whether the model is confident at predicting one of the
  shoulders/hips which is required to determine a good crop region.
  """
  return ((keypoints[0, 0, KEYPOINT_DICT['left_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE) and
          (keypoints[0, 0, KEYPOINT_DICT['left_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE))

def determine_torso_and_body_range(
    keypoints, target_keypoints, center_y, center_x):
  """Calculates the maximum distance from each keypoints to the center location.

  The function returns the maximum distances from the two sets of keypoints:
  full 17 keypoints and 4 torso keypoints. The returned information will be
  used to determine the crop size. See determineCropRegion for more detail.
  """
  torso_joints = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']
  max_torso_yrange = 0.0
  max_torso_xrange = 0.0
  for joint in torso_joints:
    dist_y = abs(center_y - target_keypoints[joint][0])
    dist_x = abs(center_x - target_keypoints[joint][1])
    if dist_y > max_torso_yrange:
      max_torso_yrange = dist_y
    if dist_x > max_torso_xrange:
      max_torso_xrange = dist_x

  max_body_yrange = 0.0
  max_body_xrange = 0.0
  for joint in KEYPOINT_DICT.keys():
    if keypoints[0, 0, KEYPOINT_DICT[joint], 2] < MIN_CROP_KEYPOINT_SCORE:
      continue
    dist_y = abs(center_y - target_keypoints[joint][0]);
    dist_x = abs(center_x - target_keypoints[joint][1]);
    if dist_y > max_body_yrange:
      max_body_yrange = dist_y

    if dist_x > max_body_xrange:
      max_body_xrange = dist_x

  return [max_torso_yrange, max_torso_xrange, max_body_yrange, max_body_xrange]

def determine_crop_region(
      keypoints, image_height,
      image_width):
  """Determines the region to crop the image for the model to run inference on.

  The algorithm uses the detected joints from the previous frame to estimate
  the square region that encloses the full body of the target person and
  centers at the midpoint of two hip joints. The crop size is determined by
  the distances between each joints and the center point.
  When the model is not confident with the four torso joint predictions, the
  function returns a default crop which is the full image padded to square.
  """
  target_keypoints = {}
  for joint in KEYPOINT_DICT.keys():
    target_keypoints[joint] = [
      keypoints[0, 0, KEYPOINT_DICT[joint], 0] * image_height,
      keypoints[0, 0, KEYPOINT_DICT[joint], 1] * image_width
    ]

  if torso_visible(keypoints):
    center_y = (target_keypoints['left_hip'][0] +
                target_keypoints['right_hip'][0]) / 2;
    center_x = (target_keypoints['left_hip'][1] +
                target_keypoints['right_hip'][1]) / 2;

    (max_torso_yrange, max_torso_xrange,
      max_body_yrange, max_body_xrange) = determine_torso_and_body_range(
          keypoints, target_keypoints, center_y, center_x)

    crop_length_half = np.amax(
        [max_torso_xrange * 1.9, max_torso_yrange * 1.9,
          max_body_yrange * 1.2, max_body_xrange * 1.2])

    tmp = np.array(
        [center_x, image_width - center_x, center_y, image_height - center_y])
    crop_length_half = np.amin(
        [crop_length_half, np.amax(tmp)]);

    crop_corner = [center_y - crop_length_half, center_x - crop_length_half];

    if crop_length_half > max(image_width, image_height) / 2:
      return init_crop_region(image_height, image_width)
    else:
      crop_length = crop_length_half * 2;
      return {
        'y_min': crop_corner[0] / image_height,
        'x_min': crop_corner[1] / image_width,
        'y_max': (crop_corner[0] + crop_length) / image_height,
        'x_max': (crop_corner[1] + crop_length) / image_width,
        'height': (crop_corner[0] + crop_length) / image_height -
            crop_corner[0] / image_height,
        'width': (crop_corner[1] + crop_length) / image_width -
            crop_corner[1] / image_width
      }
  else:
    return init_crop_region(image_height, image_width)

def crop_and_resize(image, crop_region, crop_size):
  """Crops and resize the image to prepare for the model input."""
  boxes=[[crop_region['y_min'], crop_region['x_min'],
          crop_region['y_max'], crop_region['x_max']]]
  output_image = tf.image.crop_and_resize(
      image, box_indices=[0], boxes=boxes, crop_size=crop_size)
  return output_image

def run_inference(movenet, image, crop_region, crop_size):
  """Runs model inferece on the cropped region.

  The function runs the model inference on the cropped region and updates the
  model output to the original image coordinate system.
  """
  image_height, image_width, _ = image.shape
  input_image = crop_and_resize(
    tf.expand_dims(image, axis=0), crop_region, crop_size=crop_size)
  # Run model inference.
  keypoints_with_scores = movenet(input_image)
  # Update the coordinates.
  for idx in range(17):
    keypoints_with_scores[0, 0, idx, 0] = (
        crop_region['y_min'] * image_height +
        crop_region['height'] * image_height *
        keypoints_with_scores[0, 0, idx, 0]) / image_height
    keypoints_with_scores[0, 0, idx, 1] = (
        crop_region['x_min'] * image_width +
        crop_region['width'] * image_width *
        keypoints_with_scores[0, 0, idx, 1]) / image_width
  return keypoints_with_scores

### image2gif

In [43]:
# # Load the input image.
# image_path = "C:\\Users\\SSAFY\\Videos\\Movavi Library\\bawling_o\\bawling_o_61.gif"
# image = tf.io.read_file(image_path)
# image = tf.image.decode_gif(image)
# print(image.shape)

### image2mp4

In [44]:
import cv2
import tensorflow as tf
import numpy as np

# MP4 파일 경로 설정
video_path = "C:\\Users\\SSAFY\\Desktop\\S12P11B202\\AI\\oCam\\bawling_o\\bowling_o (1).mp4"

# 비디오 캡처 객체 생성
cap = cv2.VideoCapture(video_path)

# 프레임을 저장할 리스트 초기화
frames = []

# 비디오의 모든 프레임을 읽기
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # BGR -> RGB 변환 (OpenCV는 기본적으로 BGR로 이미지를 읽음)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)

# 비디오 캡처 객체 해제
cap.release()

# 프레임 리스트가 비어 있는지 확인
if not frames:
    raise ValueError("No frames were collected. Please check the video path or file format.")

# NumPy 배열로 변환
frames_np = np.array(frames)

# 배열 모양 확인
print(f"Frames numpy array shape: {frames_np.shape}")

# 4차원 배열이 아니면 에러 발생
if len(frames_np.shape) != 4:
    raise ValueError("Frames array is not 4-dimensional. Check frame collection and dimensions.")

# TensorFlow 텐서로 변환
image = tf.convert_to_tensor(frames_np, dtype=tf.uint8)

# 텐서 모양 확인
num_frames, image_height, image_width, channels = image.shape
print(f"Number of frames: {num_frames}")
print(f"Image height: {image_height}")
print(f"Image width: {image_width}")
print(f"Channels: {channels}")


Frames numpy array shape: (145, 1240, 632, 3)
Number of frames: 145
Image height: 1240
Image width: 632
Channels: 3


In [60]:

# Load the input image.
num_frames, image_height, image_width, _ = image.shape
crop_region = init_crop_region(image_height, image_width)

output_images = []
dataset = []
bar = display(progress(0, num_frames-1), display_id=True)
for frame_idx in range(num_frames):
    keypoint_dataset = [] # keypoint 좌표가 들어가는 배열
    print(frame_idx)
    keypoints_with_scores = run_inference(
      movenet, image[frame_idx, :, :, :], crop_region,
      crop_size=[input_size, input_size])
    (keypoint_xy,output_images_input) =draw_prediction_on_image(
      image[frame_idx, :, :, :].numpy().astype(np.int32),
      keypoints_with_scores, crop_region=None,
      close_figure=True, output_image_height=300)
   
    print(keypoint_xy)
    print(len(keypoint_xy))

    for i, point in enumerate(keypoint_xy):
      print(f"index {i}: {point}")

    # print(keypoint_xy) # 키 포인트 출력 
    for keypoint_dict_idx in range(len(keypoint_xy)):
       for keypoint_xy_idx in range(len(keypoint_xy[0])):
            keypoint_dataset.append(keypoint_xy[keypoint_dict_idx][keypoint_xy_idx])

    dataset.append(keypoint_dataset)

    output_images.append(output_images_input)
    crop_region = determine_crop_region(
      keypoints_with_scores, image_height, image_width)
    bar.update(progress(frame_idx, num_frames-1))
#   output_images.append(draw_prediction_on_image(
#       image[frame_idx, :, :, :].numpy().astype(np.int32),
#       keypoints_with_scores, crop_region=None,
#       close_figure=True, output_image_height=300))

print(f"dataset :  + {dataset}")
# Prepare gif visualization.
output = np.stack(output_images, axis=0)
# to_gif(output, fps=10)
to_mp4(output,fps=30)

0
[[ 226.1176    305.61996 ]
 [ 209.01263   280.399   ]
 [ 242.87065   289.62314 ]
 [ 170.8955    286.29126 ]
 [ 258.1224    299.4724  ]
 [  78.02817   355.94324 ]
 [ 298.79477   394.0017  ]
 [  93.05387   473.91275 ]
 [ 280.8488    468.14542 ]
 [ 181.64961   422.46106 ]
 [ 233.17154   429.5125  ]
 [  66.74813   699.7663  ]
 [ 192.38902   706.1619  ]
 [  53.17575   893.688   ]
 [ 182.7625    892.95746 ]
 [  13.919653 1136.5034  ]
 [ 150.17252  1104.3888  ]]
17
index 0: [226.1176  305.61996]
index 1: [209.01263 280.399  ]
index 2: [242.87065 289.62314]
index 3: [170.8955  286.29126]
index 4: [258.1224 299.4724]
index 5: [ 78.02817 355.94324]
index 6: [298.79477 394.0017 ]
index 7: [ 93.05387 473.91275]
index 8: [280.8488  468.14542]
index 9: [181.64961 422.46106]
index 10: [233.17154 429.5125 ]
index 11: [ 66.74813 699.7663 ]
index 12: [192.38902 706.1619 ]
index 13: [ 53.17575 893.688  ]
index 14: [182.7625  892.95746]
index 15: [  13.919653 1136.5034  ]
index 16: [ 150.17252 1104.3888

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_8228\3909906277.py:162: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)


1
[[ 221.61983   301.43637 ]
 [ 205.03972   284.8421  ]
 [ 239.28949   291.5167  ]
 [ 167.56403   289.63403 ]
 [ 259.644     300.86798 ]
 [  79.70229   351.7378  ]
 [ 297.6204    392.30148 ]
 [  84.21034   469.2796  ]
 [ 275.69962   448.07837 ]
 [ 159.87996   425.40875 ]
 [ 202.4401    407.7331  ]
 [  62.498875  690.8484  ]
 [ 192.52074   700.5288  ]
 [  44.10019   896.4076  ]
 [ 177.9763    897.4512  ]
 [   1.603441 1122.2827  ]
 [ 151.5738   1102.3702  ]]
17
index 0: [221.61983 301.43637]
index 1: [205.03972 284.8421 ]
index 2: [239.28949 291.5167 ]
index 3: [167.56403 289.63403]
index 4: [259.644   300.86798]
index 5: [ 79.70229 351.7378 ]
index 6: [297.6204  392.30148]
index 7: [ 84.21034 469.2796 ]
index 8: [275.69962 448.07837]
index 9: [159.87996 425.40875]
index 10: [202.4401 407.7331]
index 11: [ 62.498875 690.8484  ]
index 12: [192.52074 700.5288 ]
index 13: [ 44.10019 896.4076 ]
index 14: [177.9763 897.4512]
index 15: [   1.603441 1122.2827  ]
index 16: [ 151.5738 1102.3702]

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_8228\3909906277.py:162: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_8228\3909906277.py:162: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)


2
[[ 221.1891     302.14603  ]
 [ 204.76256    285.71048  ]
 [ 238.8272     292.1495   ]
 [ 167.18958    289.74078  ]
 [ 259.1891     301.2903   ]
 [  79.629715   350.30417  ]
 [ 297.3859     392.2246   ]
 [  86.948296   466.53296  ]
 [ 276.68542    447.08508  ]
 [ 163.87413    425.93854  ]
 [ 206.27472    404.95593  ]
 [  63.067905   690.92395  ]
 [ 193.2969     699.57947  ]
 [  44.365376   897.67865  ]
 [ 177.43605    898.09314  ]
 [   2.0985255 1120.3848   ]
 [ 151.46233   1102.068    ]]
17
index 0: [221.1891  302.14603]
index 1: [204.76256 285.71048]
index 2: [238.8272 292.1495]
index 3: [167.18958 289.74078]
index 4: [259.1891 301.2903]
index 5: [ 79.629715 350.30417 ]
index 6: [297.3859 392.2246]
index 7: [ 86.948296 466.53296 ]
index 8: [276.68542 447.08508]
index 9: [163.87413 425.93854]
index 10: [206.27472 404.95593]
index 11: [ 63.067905 690.92395 ]
index 12: [193.2969  699.57947]
index 13: [ 44.365376 897.67865 ]
index 14: [177.43605 898.09314]
index 15: [   2.0985255 1120.

...

143
[[327.61554 434.15106]
 [325.02686 427.1634 ]
 [332.3715  429.56403]
 [307.04922 422.52823]
 [329.06433 429.27997]
 [266.58765 429.08224]
 [325.91733 458.3013 ]
 [221.97588 449.02545]
 [342.55884 489.42163]
 [179.29353 459.5435 ]
 [341.87778 446.32593]
 [214.31082 506.52844]
 [241.70427 515.1466 ]
 [254.6907  572.49255]
 [197.37573 589.55316]
 [254.77853 645.51105]
 [131.54965 634.0692 ]]
17
index 0: [327.61554 434.15106]
index 1: [325.02686 427.1634 ]
index 2: [332.3715  429.56403]
index 3: [307.04922 422.52823]
index 4: [329.06433 429.27997]
index 5: [266.58765 429.08224]
index 6: [325.91733 458.3013 ]
index 7: [221.97588 449.02545]
index 8: [342.55884 489.42163]
index 9: [179.29353 459.5435 ]
index 10: [341.87778 446.32593]
index 11: [214.31082 506.52844]
index 12: [241.70427 515.1466 ]
index 13: [254.6907  572.49255]
index 14: [197.37573 589.55316]
index 15: [254.77853 645.51105]
index 16: [131.54965 634.0692 ]


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_8228\3909906277.py:162: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)


144
[[333.89453 431.50473]
 [331.2007  425.69058]
 [337.39294 427.12613]
 [311.91495 423.91397]
 [327.85934 428.6191 ]
 [266.38486 428.73166]
 [326.36267 455.92578]
 [220.25363 449.6718 ]
 [344.9869  488.7534 ]
 [180.39839 462.9697 ]
 [360.9859  484.6427 ]
 [214.5645  505.68744]
 [242.37657 514.6474 ]
 [254.7939  570.10394]
 [196.6496  590.2855 ]
 [254.54004 645.0358 ]
 [132.54866 635.7076 ]]
17
index 0: [333.89453 431.50473]
index 1: [331.2007  425.69058]
index 2: [337.39294 427.12613]
index 3: [311.91495 423.91397]
index 4: [327.85934 428.6191 ]
index 5: [266.38486 428.73166]
index 6: [326.36267 455.92578]
index 7: [220.25363 449.6718 ]
index 8: [344.9869 488.7534]
index 9: [180.39839 462.9697 ]
index 10: [360.9859 484.6427]
index 11: [214.5645  505.68744]
index 12: [242.37657 514.6474 ]
index 13: [254.7939  570.10394]
index 14: [196.6496 590.2855]
index 15: [254.54004 645.0358 ]
index 16: [132.54866 635.7076 ]


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_8228\3909906277.py:162: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)


TypeError: can only concatenate str (not "list") to str